## Autor: Antunes Dantas

O objetivo deste documento é mostrar como implementar um motor de busca que usa algoritmos de rankeamento de documentos mais eficazes como o TF, TF-IDF e o BM25.
Primeiro, vamos importar as bibliotecas necessárias para a implementação do algoritmo.

In [1]:
import pandas as pd
import nltk
import numpy as np
from functools import reduce
import math


Agora, será necessário configurar o básico para a criação do índice invertido. A minha estratégia aqui foi de criar
uma segunda estrutura de dados. Essa estrutura, chamada de TF, guardará o term frequency para cada documento. Assim, quando for preciso acessar tal valor, poderei fazer em tempo constante. Além disso será necessário ter cuidado para tratar os valores NA (valores inexistentes) para evitar erros durante a execução dos algoritmos.

In [20]:
NOME_DO_ARQUIVO = 'estadao_noticias_eleicao.csv'
indice_invertido = dict()
dados = pd.read_csv(NOME_DO_ARQUIVO)
dados = dados.replace(np.nan, '', regex=True)
tf = {}

Para a geração do índice invertido, a única mudança é que agora calcularemos o term frequency de cada termo do documento.

In [22]:
def geraIndice():
    for i, row in dados.iterrows():
        
        titulo = (word.lower() for word in (nltk.word_tokenize(row['titulo'])))
        subtitulo = (word.lower() for word in (nltk.word_tokenize(row['subTitulo'])))
        conteudo = (word.lower() for word in (nltk.word_tokenize(row['conteudo'])))
        palavras_divididas = list(titulo) + list(subtitulo) + list(conteudo)
        tfDoc = {}
        for palavra in palavras_divididas:
            if palavra not in indice_invertido:
                indice_invertido[palavra] = set([row['idNoticia']])
            else:
                indice_invertido[palavra].add(row['idNoticia'])
            if palavra not in tfDoc:
                tfDoc[palavra] = 0
            tfDoc[palavra] += 1
        tf[row['idNoticia']] = tfDoc

O modelo binário consiste de uma consulta onde os documentos retornados são aqueles que contém todos os termos da busca. Não é muito eficaz, visto que não realiza nenhum rankeamento.

In [94]:
def modeloBinario(termos):
    indice = []
    for termo in termos:
        indice.append(indice_invertido[termo])
    return list(set.intersection(*indice))

In [7]:
def generateQueryTF(query):
    tf = {}
    for term in query:
        if term not in tf:
            tf[term] = 0
        tf[term] += 1
    return tf

Já o modelo TF usa o term frequency de cada termo no documento para mostrar os documentos onde o termo aparece mais. Ou seja, quanto mais vezes o termo aparece no documento e na query, mais ele será rankeado para cima.

In [28]:
def modeloTF(termos):
    documentos = modeloBinario(termos)
    documentsTF = []
    for idDoc in documentos:
        documentWeight = 0
        for termo in termos:
            docTF = tf[idDoc][termo] if (termo in tf[idDoc]) else 0
            qTF = termos.count(termo)
            documentWeight += docTF * qTF
            
        documentsTF.append((idDoc, documentWeight))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    ids = []
    for tupla in documentsTF:
        ids.append(tupla[0])
    return ids

O modelo TF-IDF leva em conta além do term frequency do documento, a quantidade de vezes que aquele termo apareceu no documento. Assim, quando um termo aparece muitas vezes no documento, ele perde relevância. É especialmente útil para evitar que stop words tenham um poder de rankeamento grande no documento.

In [29]:
def modeloIDF(termos):
    documentos = modeloBinario(termos)
    documentsTF = []
    for idDoc in documentos:
        documentWeight = 0
        for termo in termos:
            docTF = tf[idDoc][termo] if (termo in tf[idDoc]) else 0
            qTF = termos.count(termo)
            idf = math.log(dados.titulo.size/len(indice_invertido[termo]))
            documentWeight += docTF * qTF * idf
            
        documentsTF.append((idDoc, documentWeight))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    ids = []
    for tupla in documentsTF:
        ids.append(tupla[0])
    return ids

O BM25 aprimora o TF-IDF. Ele ajuda na normalização da importância de um termo na busca, assim, a nós conseguimos controlar a curva de crescimento da importância de um termo.

In [50]:
def modeloBM25(termos):
    documentos = modeloBinario(termos)
    documentsTF = []
    k = 5
    for idDoc in documentos:
        documentWeight = 0
        for termo in termos:
            docTF = tf[idDoc][termo] if (termo in tf[idDoc]) else 0
            qTF = termos.count(termo)
            idf = math.log(dados.titulo.size/len(indice_invertido[termo]))
            bm25 = ((k + 1)*(docTF)) / (docTF + k)
            documentWeight += docTF * qTF * idf * bm25
            
        documentsTF.append((idDoc, documentWeight))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    ids = []
    for tupla in documentsTF:
        ids.append(tupla[0])
    return ids

Para avaliar o resultado da implementação, vamos realizar um conjunto de buscas e compará-las com um um gabarito fornecido.

In [103]:
queries = ['segundo turno', 'lava jato', 'projeto de lei', 'compra de voto', 'ministério público']
resultados = {
    'modeloBinario' : [],
    'tf' : [],
    'idf' : [],
    'bm25' : []
}

for query in queries:
    termos = query.split()
    resultados['modeloBinario'].append(str(modeloBinario(termos)))
    resultados['tf'].append(str(modeloTF(termos)))
    resultados['idf'].append(str(modeloIDF(termos)))
    resultados['bm25'].append(str(modeloBM25(termos)))

In [101]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [104]:
gabarito = pd.read_csv('gabarito.csv')

print(mapk(gabarito.busca_binaria, resultados['modeloBinario'], k = 5))
print(mapk(gabarito.tf, resultados['tf'], k = 5))
print(mapk(gabarito.tfidf, resultados['idf'], k = 5))
print(mapk(gabarito.bm25, resultados['bm25'], k = 5))

0.96
0.96
0.8699999999999999
0.8620000000000001


Analisando o resultado dos modelos juntamente com o gabarito, é possível perceber que a implementação obteve um grande nível de acuracia com o gabarito.
Observando o resultado por modelo, vemos que o TF e o Binário possuem a maior nota. Isso deve-se ao fato de que ambos os modelos não possuem um rankeamento complexo.
O BM25, por exemplo, pode ter o resultado variado de acordo com o parâmetro k passado, e isso pode influenciar no resultado.

É importante ressaltar que o modelo TF obteve uma classificação tão boa devido ao escopo do nosso índice. Como não temos ma grande quantidade de documentos, a quantidade de documentos em que o termo apareceu não causa grande impacto nos resultados, porém é esperado que em grandes conjuntos de dados esse valor afete (melhore) a qualidade dos resultados.